<a href="https://colab.research.google.com/github/mint-rin/SeSAC/blob/main/topic_modeling_3_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# 경고 메시지 끄기
warnings.filterwarnings(action='ignore')

# 참고) 다시 출력하게 하기
# warnings.filterwarnings(action='default')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/02_대학생/05_대외활동/230828_SeSAC_핀테크데이터분석가과정/06_NLP/231018/review.csv',
                 sep = '\t')

In [4]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 토큰화 및 데이터 전처리

In [5]:
# !pip install konlpy

In [6]:
# 한국어 형태소 분석
from konlpy.tag import Okt
tokenizer = Okt()

df['morphs'] = None
for i, row in df.iterrows():
  df['morphs'][i] = ' '.join(tokenizer.morphs(row['document']))

df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 .. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만...


In [7]:
import re

# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")

def preprocess_remove(text):
  text = re.sub(remove_except_ko, ' ', text).strip()
  return text

df['morphs'] = df['morphs'].map(lambda x : preprocess_remove(x))
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...


# 불용어 처리

In [8]:
# 불용어 처리
stop_word = ['을', '은', '가']

def preprocess(text):
  text = text.split()
  text = [i for i in text if len(i)>1]
  text = [i for i in text if i not in stop_word]
  return text

In [9]:
# 토큰화 + 토큰리스트 생성
def make_tokens(df):
  df['tokens'] = ''
  for i, row in df.iterrows():
    if i % 1000 == 0:
      print(i, '/', len(df))
    token = preprocess(df['morphs'][i])
    df['tokens'][i] = ' '.join(token)
  return df

df = make_tokens(df)

0 / 10000
1000 / 10000
2000 / 10000
3000 / 10000
4000 / 10000
5000 / 10000
6000 / 10000
7000 / 10000
8000 / 10000
9000 / 10000


In [10]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,포스터 보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했던 커스틴 ...


In [11]:
neg = (df[df['label']==0]).reset_index(drop = True)
pos = (df[df['label']==1]).reset_index(drop = True)
print('neg:', len(neg))
print('pos:', len(pos))

neg: 5021
pos: 4979


In [12]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,포스터 보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했던 커스틴 ...


In [13]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

tokenized_docs = neg['tokens'].apply(lambda x : x.split())

id2word = corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10  # 토픽 개수 - 변경 가능
lda = LdaModel(corpus = corpus_TFIDF,
               id2word = id2word,
               num_topics = n,
               random_state = 100)

for t in lda.print_topics():
  print(t)

(0, '0.008*"영화" + 0.005*"졸작" + 0.004*"점도" + 0.004*"너무" + 0.004*"시간" + 0.004*"이다" + 0.004*"진짜" + 0.004*"스토리" + 0.004*"아깝다" + 0.004*"감독"')
(1, '0.009*"영화" + 0.004*"너무" + 0.004*"코미디" + 0.003*"기세" + 0.003*"그냥" + 0.003*"보면" + 0.003*"내용" + 0.003*"역시" + 0.003*"액션" + 0.003*"정말"')
(2, '0.007*"영화" + 0.006*"없다" + 0.005*"재미없다" + 0.005*"재미" + 0.005*"최악" + 0.004*"아깝다" + 0.003*"보다" + 0.003*"진짜" + 0.003*"봤다" + 0.003*"정말"')
(3, '0.006*"최악" + 0.006*"영화" + 0.004*"지루하다" + 0.004*"그대로" + 0.003*"점수" + 0.003*"하하" + 0.003*"이상한" + 0.003*"제목" + 0.002*"인생" + 0.002*"아마"')
(4, '0.006*"영화" + 0.004*"ㅋㅋㅋㅋ" + 0.004*"없는" + 0.004*"ㅋㅋ" + 0.004*"에서" + 0.004*"이건" + 0.003*"별루" + 0.003*"없고" + 0.003*"재미" + 0.003*"였다"')
(5, '0.006*"영화" + 0.004*"평점" + 0.003*"나옴" + 0.003*"너무" + 0.003*"이런" + 0.003*"라고" + 0.003*"알바" + 0.003*"지루해" + 0.003*"에서" + 0.003*"시간"')
(6, '0.003*"영화" + 0.003*"으리" + 0.003*"따라" + 0.002*"취향" + 0.002*"충격" + 0.002*"지는" + 0.002*"섹스" + 0.002*"하네" + 0.002*"원래" + 0.002*"있었다"')
(7, '0.008*"별로" + 0.006*"영화" + 0.005*"같은"

# 시각화

In [14]:
!pip install pyLDAvis==3.4.1

## n = 10

In [15]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word)
pyLDAvis.display(vis)

## n = 3

In [16]:
n = 3   # 토픽 개수 - 변경 가능

lda = LdaModel(corpus = corpus_TFIDF,
               id2word = id2word,
               num_topics = n,
               random_state = 100)

for t in lda.print_topics():
  print(t)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_TFIDF, id2word)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.005*"영화" + 0.005*"쓰레기" + 0.003*"진짜" + 0.003*"별로" + 0.003*"이런" + 0.003*"너무" + 0.002*"스토리" + 0.002*"아깝다" + 0.002*"감독" + 0.002*"시간"')
(1, '0.006*"영화" + 0.002*"정말" + 0.002*"그냥" + 0.002*"내용" + 0.002*"에서" + 0.002*"평점" + 0.002*"너무" + 0.002*"까지" + 0.002*"으로" + 0.002*"시간"')
(2, '0.005*"영화" + 0.003*"최악" + 0.002*"보다" + 0.002*"정말" + 0.002*"재미없다" + 0.002*"ㅡㅡ" + 0.002*"그냥" + 0.002*"이건" + 0.002*"없다" + 0.002*"이다"')
